In [ ]:
import sys

# ⬇️ Adjust this to the folder that contains the `edgar` package (NOT the edgar folder itself)
# Example if your repo is C:\Python\edgartools and inside it is "edgar/..."
sys.path.append(r"C:\Python\edgartools")  # <-- change if needed

from edgar import Company, XBRL, set_identity
from edgar.xbrl.analysis import metrics

# Use a real email for SEC user-agent
set_identity("kamboj.r@gmail.com")


In [ ]:
class NormalizedStatements:
    def __init__(self, stmts):
        # Helper to normalize one statement attr (call if needed)
        def norm_one(name):
            if not hasattr(stmts, name):
                return None
            attr = getattr(stmts, name)
            return attr() if callable(attr) else attr

        # Balance sheet & income statement (these exist in your metrics code)
        self.balance_sheet = norm_one("balance_sheet")
        self.income_statement = norm_one("income_statement")

        # Cash flow can have different names depending on edgartools version
        cf = None
        for cand in ("cash_flow", "cash_flow_statement", "cashflow", "statement_of_cash_flows"):
            if hasattr(stmts, cand):
                attr = getattr(stmts, cand)
                cf = attr() if callable(attr) else attr
                break

        self.cash_flow = cf  # may be None if not available


class NormalizedXBRL:
    def __init__(self, x):
        # Wrap original XBRL but replace .statements with normalized version
        self.statements = NormalizedStatements(x.statements)



In [ ]:
from edgar.xbrl.xbrl import XBRL

# Create company object for Microsoft
msft = Company("MSFT")

# Get all 10-K filings
msft_10k_filings = msft.get_filings(form="10-K")

print("Total MSFT 10-K filings returned:", len(msft_10k_filings))

# Pick one filing (usually index 0 is the latest; if your version supports latest(), you can use that)
try:
    filing = msft_10k_filings.latest(1)
except AttributeError:
    filing = msft_10k_filings[0]

print("Using filing:", filing)

# Build XBRL object from this filing
xbrl = XBRL.from_filing(filing)
xbrl


Total MSFT 10-K filings returned: 32
Using filing: Filing(company='MICROSOFT CORP', cik=789019, form='10-K', filing_date='2025-07-30', accession_no='0000950170-25-100235')


╭───────────────────────────────────────────────── XBRL Document ─────────────────────────────────────────────────╮
│ MICROSOFT CORPORATION (MSFT) • CIK 0000789019                                                                   │
│                                                                                                                 │
│          Form:  10-K                                                                                            │
│ Fiscal Period:  Fiscal Year 2025 (ended Jun 30, 2025)                                                           │
│          Data:  1,829 facts • 441 contexts                                                                      │
│                                                                                                                 │
│ Available Data Coverage:                                                                                        │
│   Annual: FY 2025                                                     

In [ ]:
nx = NormalizedXBRL(xbrl)

altman = AltmanZScore(nx)
beneish = BeneishMScore(nx)
piotroski = PiotroskiFScore(nx)

results = {
    "Altman Z": altman.calculate(),
    "Beneish M": beneish.calculate(),
    "Piotroski F": piotroski.calculate()
}

results



AttributeError: 'Statement' object has no attribute 'periods'